# **UMD CS courses: What do the reviews say?**
*CMSC320 <br>
Professor Morawski, Fall 2022* <br>
*Adam Yang (ayang115@umd.edu)*



In [2]:
import planetterp as pt
import pandas as pd

Objectives:
* clean data to have professor, course number, grade data, no credits, title, average score
* create sentiment analysis for reviews
* compare sentiment to reviews
* compare grade to reviews
* rank sentiment by rating
* rank grade by rating

Questions:
1. What deems a course to be good?
2. Which professors are the best for each course?
3. What kind of sentiments are present for highly rated professors

In [3]:
ugrad_courses = pd.DataFrame(pt.courses(department="CMSC", reviews=True))
ugrad_courses.to_csv("cmsc_courses")

ugrad_courses = ugrad_courses[ugrad_courses["course_number"] < "500"]
ugrad_courses["sub_code"] = ugrad_courses["course_number"].apply(lambda x: x[3] if len(x) > 3 else 0) 

In [4]:
ugrad_courses.head()

,average_gpa,professors,reviews,department,course_number,name,title,credits,description,is_recent,sub_code
0,3.199664,"[Atif Memon, Adam Porter, Charles Song, Vibha ...","[{'professor': 'Adam Porter', 'course': 'CMSC4...",CMSC,436,CMSC436,Programming Handheld Systems,3.0,<b>Prerequisite:</b> Minimum grade of C- in CM...,True,0
1,2.672616,"[Amol Deshpande, Nicholas Roussopoulos, Peter ...","[{'professor': 'Amol Deshpande', 'course': 'CM...",CMSC,424,CMSC424,Database Design,3.0,<b>Prerequisite:</b> Minimum grade of C- in CM...,True,0
2,3.461364,"[Charles Kassir, Amy Vaillancourt, Alyssa Neun...","[{'professor': 'Corie Brown', 'course': 'CMSC1...",CMSC,100,CMSC100,Bits and Bytes of Computer and Information Sci...,1.0,<b>Restriction:</b> For first time freshmen an...,True,0
4,2.449417,"[Alan Sussman, Larry Herman, Neil Spring, Nels...","[{'professor': 'Nelson Padua-Perez', 'course':...",CMSC,216,CMSC216,Introduction to Computer Systems,4.0,<b>Prerequisite:</b> Minimum grade of C- in CM...,True,0
5,2.746843,"[Chau-Wen Tseng, Larry Herman, Dave Levin, Mic...","[{'professor': 'Jeffrey Foster', 'course': 'CM...",CMSC,330,CMSC330,Organization of Programming Languages,3.0,<b>Prerequisite:</b> Minimum grade of C- in CM...,True,0


**Description of Data**

For this project, data regarding UMD courses is used. Using the [Planetterp API](https://planetterp.com/api/#tag/Search/operation/Search), course information was extracted into a dataframe. <br>
<br>
Course Information:<br>
Grouped by term, course information includes instructors, department, course code, course title, number of credits, prerequisites, course recency, and a dictionary containing reviews. For the purposes of this project, only undergraduate computer science courses (courses with heading CMSC and coded with value 500 or less) were studied.